In [ ]:
import torch
from torch_geometric.datasets import MovieLens1M, MovieLens100K


# Load MovieLens 1M dataset (adjust root if needed) push test
#dataset = MovieLens1M(root='/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/temp_pyg')
dataset= MovieLens100K(root="/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/temp_pyg")
print(dataset[0])


In [1]:
import torch

# Parameters
batch_size = 2
num_negatives = 2
num_users = 3
num_items = 3
dim = 2

# Embeddings for users and items (3 users, 3 items, embedding dimension 2)
user_embedding = torch.tensor([[0.1, 0.2], [0.3, 0.4], [0.5, 0.6]])
item_embedding = torch.tensor([[0.7, 0.8], [0.9, 1.0], [1.1, 1.2]])

# h_index and t_index based on the edges and negative samples
# Each row represents a batch entry, and each column is a negative sample
# Here, `0` and `1` are valid head indices (users), and we corrupt them by keeping valid and invalid tails
h_index = torch.tensor([
    [0, 0, 0],  # For first edge (0, 0, 3) with two negatives
    [1, 2, 0]   # For second edge (1, 0, 4) with two negatives
])
t_index = torch.tensor([
    [3, 4, 5],  # Original (0, 0, 3), and corrupted tails [4, 2]
    [4, 4, 4]   # Original (1, 0, 4), and corrupted tails [5, 0]
])

# Gather head node embeddings
# (num_nodes, dim)
index_temp = h_index.unsqueeze(-1).expand(-1, -1, user_embedding.shape[-1])
h_embeddings = user_embedding.unsqueeze(0).expand(batch_size,-1,-1).gather(1, h_index.unsqueeze(-1).expand(-1, -1, dim))

# Adjust `t_index` to map to item IDs by subtracting `num_users`
index_temp = (t_index - num_users).clamp(min=0)
t_embeddings = item_embedding.unsqueeze(0).expand(batch_size,-1,-1).gather(1, index_temp.unsqueeze(-1).expand(-1, -1, dim))

print("User Embedding Tensor:\n", user_embedding)
print("Item Embedding Tensor:\n", item_embedding)
print("Head Embeddings:\n", h_embeddings)
print("Tail Embeddings:\n", t_embeddings)
print("size head Embeddings:\n", h_embeddings.shape)
print("size Tail Embeddings:\n", t_embeddings.shape)


User Embedding Tensor:
 tensor([[0.1000, 0.2000],
        [0.3000, 0.4000],
        [0.5000, 0.6000]])
Item Embedding Tensor:
 tensor([[0.7000, 0.8000],
        [0.9000, 1.0000],
        [1.1000, 1.2000]])
Head Embeddings:
 tensor([[[0.1000, 0.2000],
         [0.1000, 0.2000],
         [0.1000, 0.2000]],

        [[0.3000, 0.4000],
         [0.5000, 0.6000],
         [0.1000, 0.2000]]])
Tail Embeddings:
 tensor([[[0.7000, 0.8000],
         [0.9000, 1.0000],
         [1.1000, 1.2000]],

        [[0.9000, 1.0000],
         [0.9000, 1.0000],
         [0.9000, 1.0000]]])
size head Embeddings:
 torch.Size([2, 3, 2])
size Tail Embeddings:
 torch.Size([2, 3, 2])


In [ ]:
data = dataset[0]
plot_ratings_vs_time(data)
#print (data)
#print (data['user', 'rates', 'movie'].rating[:20])
#print (graph["movie"].x[:10,:])


In [9]:
import torch
t_ranking = torch.zeros(5)
h_ranking = torch.zeros(6)
rankings1 = [t_ranking, h_ranking]
print(rankings1)
rankings2 = torch.cat([t_ranking, h_ranking], dim=0)
print(rankings2)


[tensor([0., 0., 0., 0., 0.]), tensor([0., 0., 0., 0., 0., 0.])]
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [8]:

user_features = graph["user"].x
print (movies_features[:10])

tensor([[0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])


In [8]:
import os
os.chdir("/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec") 

In [ ]:
%run script/pretrain.py -c config/recommender/pretrain_notebook.yaml --gpus [0]  /itet-stor/trachsele/net_scratch/tl4rec/model_outputs/logs/Gru-Ultra/LastFM/2025-02-27-08-44-41/model_epoch_8.pth

In [ ]:
%run script/run.py -c config/notebook_cfg.yaml --dataset LastFM --epochs 10 --bpe 10 --gpus "[0]" --ckpt /itet-stor/trachsele/net_scratch/tl4rec/ckpts/BookX.pth

/itet-stor/trachsele/net_scratch/conda_envs/ba_bugfix/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
10:04:47   Random seed: 1024
10:04:47   Config file: config/notebook_cfg.yaml
10:04:47   {'checkpoint': '/itet-stor/trachsele/net_scratch/tl4rec/ckpts/BookX.pth',
 'dataset': {'class': 'LastFM',
             'root': '/itet-stor/trachsele/net_scratch/tl4rec/model_outputs/data'},
 'model': {'backbone_model': {'embedding_item': {'dropout_rate': 0.1,
                                                 'hidden_dims': [32, 32],
                                                 'use_dropout': False,
                                                 'use_layer_norm': False},
                              'embedding_user': {'dropout_rate': 0.1,
                                                 'hidden_dims': [32, 32],


We will evaluate vs 1000 negatives
edge_attr.shape = torch.Size([79434, 1])


In [2]:
%run script/run.py -c config/recommender/notebook_cfg.yaml --dataset Yelp18 --epochs 10 --bpe 50000 --gpus "[0]" --ckpt null

Exception: File `'script/run.py'` not found.

In [ ]:



%run script/run.py -c config/recommender/notebook_cfg.yaml --dataset Yelp18 --epochs 4 --bpe 10 --gpus "[0]" --ckpt /itet-stor/trachsele/net_scratch/tl4rec/ckpts/Beauty_Games_2025-01-16_04-20.pth

In [ ]:
import numpy as np
import torch

print(np.__version__)  # Check NumPy version
print(torch.__version__)  # Check PyTorch version
print(torch.cuda.is_available())  # Check if CUDA is available


In [12]:
import sqlite3
import pandas as pd

# Define the database file path
DB_FILE = "//itet-stor/trachsele/net_scratch/tl4rec/model_outputs/result.db"

# Connect to the database
with sqlite3.connect(DB_FILE) as conn:
    # Read the table into a Pandas DataFrame
    query = "SELECT * FROM results"
    df = pd.read_sql_query(query, conn)

# Display the DataFrame
print(df)


             ckpt   dataset  epochs   bpe FT valid_mr  valid_mrr  \
0             ETE    LastFM       8   620  1     None   0.500381   
1   Beau_Epin.pth    LastFM       1   381  1     None   0.387966   
2   Beau_Epin.pth  Epinions       1  2565  1     None   0.332074   
3   Beau_Epin.pth     BookX       1  2657  1     None   0.302614   
4   Beau_Epin.pth      Ml1m       1  3506  1     None   0.631661   
5             ETE  Epinions       8  4168  1     None   0.374576   
6             ETE     BookX       8  4317  1     None   0.343489   
7             ETE      Ml1m       8  5697  1     None   0.677769   
8      LastFM.pth    LastFM       0   381  1     None   0.494322   
9        Ml1m.pth    LastFM       0   381  1     None   0.484531   
10   Epinions.pth    LastFM       0   381  1     None   0.490242   
11   Epinions.pth      Ml1m       0  3506  1     None   0.608030   
12     LastFM.pth      Ml1m       0  3506  1     None   0.613056   
13       Ml1m.pth      Ml1m       0  3506  1    

In [3]:
import sqlite3
import pandas as pd

# Define the database file path
DB_FILE = "/itet-stor/trachsele/net_scratch/tl4rec/model_outputs/results.db"

# Connect to the database
with sqlite3.connect(DB_FILE) as conn:
    # Read only the rows where ckpt = 'ETE'
    query = "SELECT * FROM experiments WHERE ckpt = 'ETE'"
    df_ete = pd.read_sql_query(query, conn)

# Display the DataFrame
print(


Executing: ALTER TABLE results RENAME COLUMN test_ndcg_10 TO "test_ndcg@10";
Executing: ALTER TABLE results RENAME COLUMN test_hits_1 TO "test_hits@1";
Executing: ALTER TABLE results RENAME COLUMN test_hits_3 TO "test_hits@3";
Executing: ALTER TABLE results RENAME COLUMN test_hits_10 TO "test_hits@10";
Executing: ALTER TABLE results RENAME COLUMN valid_ndcg_10 TO "valid_ndcg@10";
Executing: ALTER TABLE results RENAME COLUMN valid_hits_1 TO "valid_hits@1";
Executing: ALTER TABLE results RENAME COLUMN valid_hits_3 TO "valid_hits@3";
Executing: ALTER TABLE results RENAME COLUMN valid_hits_10 TO "valid_hits@10";
Columns renamed successfully.


In [2]:
import sqlite3
from pathlib import Path

# Define the database file path
DB_FILE = "//itet-stor/trachsele/net_scratch/tl4rec/model_outputs/result.db"

# Ensure the directory exists before writing
Path(DB_FILE).parent.mkdir(parents=True, exist_ok=True)

# Define the schema for the results table
CREATE_TABLE_QUERY = """
CREATE TABLE IF NOT EXISTS results (
    ckpt TEXT,
    dataset TEXT,
    epochs INTEGER,
    bpe INTEGER,
    FT TEXT,
    valid_mr REAL,
    valid_mrr REAL,
    valid_hits_1 REAL,
    valid_hits_3 REAL,
    valid_hits_10 REAL,
    valid_ndcg_10 REAL,
    test_mr REAL,
    test_mrr REAL,
    test_hits_1 REAL,
    test_hits_3 REAL,
    test_hits_10 REAL,
    test_ndcg_10 REAL,
    valid_ndcg_20 REAL,
    test_ndcg_20 REAL
);
"""

# Create the database and table
with sqlite3.connect(DB_FILE) as conn:
    cursor = conn.cursor()
    cursor.execute(CREATE_TABLE_QUERY)
    conn.commit()

print(f"Database created successfully at {DB_FILE} with an empty 'results' table.")


Database created successfully at //itet-stor/trachsele/net_scratch/tl4rec/model_outputs/result.db with an empty 'results' table.


In [11]:
import sqlite3
import pandas as pd

# Define the database file path
DB_FILE = "/itet-stor/trachsele/net_scratch/tl4rec/model_outputs/result.db"

# Connect to the database and remove duplicates
with sqlite3.connect(DB_FILE) as conn:
    cursor = conn.cursor()
    
    # Delete duplicate rows while keeping only the earliest entry for the specified condition
    cursor.execute("""
        DELETE FROM results
        WHERE rowid NOT IN (
            SELECT MIN(rowid)
            FROM results
            WHERE ckpt = 'Beau_Epin.pth' 
            AND dataset = 'Epinions' 
            AND epochs = 1 
            AND bpe = 2565
        )
        AND ckpt = 'Beau_Epin.pth' 
        AND dataset = 'Epinions' 
        AND epochs = 1 
        AND bpe = 2565;
    """)

    # Commit the changes
    conn.commit()

    # Read the updated table
    df = pd.read_sql_query("SELECT * FROM results", conn)

# Display the updated DataFrame
print(df)


             ckpt   dataset  epochs   bpe FT valid_mr  valid_mrr  \
0             ETE    LastFM       8   620  1     None   0.500381   
1   Beau_Epin.pth    LastFM       1   381  1     None   0.387966   
2   Beau_Epin.pth  Epinions       1  2565  1     None   0.332074   
3   Beau_Epin.pth     BookX       1  2657  1     None   0.302614   
4   Beau_Epin.pth      Ml1m       1  3506  1     None   0.631661   
5             ETE  Epinions       8  4168  1     None   0.374576   
6             ETE     BookX       8  4317  1     None   0.343489   
7             ETE      Ml1m       8  5697  1     None   0.677769   
8      LastFM.pth    LastFM       0   381  1     None   0.494322   
9        Ml1m.pth    LastFM       0   381  1     None   0.484531   
10   Epinions.pth    LastFM       0   381  1     None   0.490242   
11   Epinions.pth      Ml1m       0  3506  1     None   0.608030   
12     LastFM.pth      Ml1m       0  3506  1     None   0.613056   
13       Ml1m.pth      Ml1m       0  3506  1    